Interactive maps provide a dynamic user experience, allowing for real-time zooming, panning, and exploration of geospatial data layers. In contrast, Matplotlib's static figures, while useful for publication-quality visuals, lack the ability to explore data or to easily integrate multiple layers of information. Additionally, interactive maps can be easily integrated withing websites and are ideal for collaborative projects and public dissemination of geospatial research findings.

In this tutorial we will create an interactive map to explore different land covers across the state of Kansas. We will use the `folium` Python library, which is a wrapper around the popular Leaflet library implemented Javascript.

In this tutorial we will learn:

- How to add raster layers to an interactive map
- How to add vector layers to an interactive map


In [1]:
# Import modules
import ee
import folium
import numpy as np
from matplotlib import colors, colormaps


## Define some helper functions

Let's define some functions to avoid repeating the following code everytime we want to create a raster image or every time we want to specify a given colormap for our map.

In [2]:
def get_raster_map(ee_object, vis, name):
    """
    Function that retrieves raster map from GEE and applies some style.
    """
    if isinstance(ee_object, ee.Image):
        layer = folium.TileLayer(ee_object.getMapId(vis)['tile_fetcher'].url_format,
                           name=name,
                           overlay=True,
                           control=True,
                           attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>')
        
    return layer



def get_vector_map(ee_object,vis,name):
    """
    Function that retrieves vector map from GEE and applies some style.
    """
    if isinstance(ee_object, ee.FeatureCollection):
        layer = folium.GeoJson(ee_object.getInfo(),
                               name=name,
                               style_function=lambda feature: {
                                   'fillColor': vis['face_color'],
                                   'color': vis['edge_color'],
                                   'weight': vis['line_width']})
    return layer



In [6]:
# Trigger the authentication flow.
#ee.Authenticate(auth_mode='localhost')

# Initialize the library.
#ee.Initialize(project='ee-ializarazos')

## State land cover

In [7]:
# Read US states
US_states = ee.FeatureCollection("TIGER/2018/States")

# Select Kansas
region = US_states.filter(ee.Filter.inList('NAME',['Kansas']))


In [8]:
# Land use for 2021
land_use = ee.ImageCollection('USDA/NASS/CDL')\
             .filter(ee.Filter.date('2020-01-01', '2021-12-31')).first().clip(region)

# Select cropland layer
cropland = land_use.select('cropland')


In [9]:
# Get layer metadata
info = cropland.getInfo()

# Remove comment to print the entire information (output is long!)
# print(info)
print(info.keys())

dict_keys(['type', 'bands', 'version', 'id', 'properties'])


In [10]:
# Get land cover names, values, and colors from metadata

class_names_str = info['properties']['cropland_class_names']
class_values_str = info['properties']['cropland_class_values']
class_colors_str = info['properties']['cropland_class_palette']


In [11]:
print(type(class_names_str), type(class_values_str), type(class_colors_str))

<class 'str'> <class 'str'> <class 'str'>


In [12]:
class_names = [s.strip() for s in class_names_str.split(",")]
class_values = [int(s.strip()) for s in class_values_str.split(",")]
class_colors = [s.strip() for s in class_colors_str.split(",")]

print(len(class_names), len(class_values), len(class_colors))

134 134 134


In [13]:
n = min(len(class_names), len(class_values), len(class_colors))

class_props = {
    name: {"value": v, "color": c}
    for name, v, c in zip(class_names[:n], class_values[:n], class_colors[:n])
}

In [14]:
class_props

{'Background': {'value': 0, 'color': '000000'},
 'Corn': {'value': 1, 'color': 'ffd400'},
 'Cotton': {'value': 2, 'color': 'ff2626'},
 'Rice': {'value': 3, 'color': '00a9e6'},
 'Sorghum': {'value': 4, 'color': 'ff9e0f'},
 'Soybeans': {'value': 5, 'color': '267300'},
 'Sunflower': {'value': 6, 'color': 'ffff00'},
 'Peanuts': {'value': 10, 'color': '70a800'},
 'Tobacco': {'value': 11, 'color': '00af4d'},
 'Sweet Corn': {'value': 12, 'color': 'e0a60f'},
 'Pop or Orn Corn': {'value': 13, 'color': 'e0a60f'},
 'Mint': {'value': 14, 'color': '80d4ff'},
 'Barley': {'value': 21, 'color': 'e2007f'},
 'Durum Wheat': {'value': 22, 'color': '8a6453'},
 'Spring Wheat': {'value': 23, 'color': 'd9b56c'},
 'Winter Wheat': {'value': 24, 'color': 'a87000'},
 'Other Small Grains': {'value': 25, 'color': 'd69dbc'},
 'Dbl Crop WinWht/Soybeans': {'value': 26, 'color': '737300'},
 'Rye': {'value': 27, 'color': 'ae017e'},
 'Oats': {'value': 28, 'color': 'a15889'},
 'Millet': {'value': 29, 'color': '73004c'},
 

In [15]:
# Create self masks for each land cover, so that only 
# the pixels for this land cover visible

corn = cropland.eq(class_props['Corn']['value']).selfMask()
sorghum = cropland.eq(class_props['Sorghum']['value']).selfMask()
soybeans = cropland.eq(class_props['Soybeans']['value']).selfMask()
wheat = cropland.eq(class_props['Winter Wheat']['value']).selfMask()
grassland = cropland.eq(class_props['Grass/Pasture']['value']).selfMask()


In [16]:
# Initialize map centered at a specific location and zoom level
m = folium.Map(location=[38, -98], zoom_start=7)

get_raster_map(corn, {'palette':[class_props['Corn']['color']]}, 'Corn').add_to(m)
get_raster_map(sorghum, {'palette':[class_props['Sorghum']['color']]}, 'Sorghum').add_to(m)
get_raster_map(soybeans, {'palette':[class_props['Soybeans']['color']]}, 'Soybeans').add_to(m)
get_raster_map(wheat, {'palette':[class_props['Winter Wheat']['color']]}, 'Wheat').add_to(m)
get_raster_map(grassland, {'palette':[class_props['Grass/Pasture']['color']]}, 'Grassland').add_to(m)

# Add vector layers
get_vector_map(region, vis={'face_color':'#00FFFF00',
                     'edge_color':'black',
                     'line_width':2}, name='State boundary').add_to(m)

#get_vector_map(counties, vis={'face_color':'#00FFFF00',
                     #'edge_color':'grey',
                     #'line_width':1}, name='County boundary').add_to(m)


# Add a layer control panel to the map.
m.add_child(folium.LayerControl())

# Display the map.
display(m)
